In [1]:
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import sys

In [2]:
url_poro = 'https://raw.githubusercontent.com/sherkidanil/skoltech_st_works/main/porosity_spe10.csv'
url_perm = 'https://raw.githubusercontent.com/sherkidanil/skoltech_st_works/main/permeability_spe10.csv'

In [3]:
df = pd.read_csv(url_poro, delimiter=';')
poro_tensor = df.to_numpy()
poro_tensor = np.reshape(poro_tensor, [60,220,85])

In [4]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:

import tensorflow as t
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
import matplotlib.pyplot as pt

In [6]:
poro_tensor.shape

(60, 220, 85)

In [7]:
data_tens = []
for i in range(poro_tensor.shape[0]):
  data_tens.append(poro_tensor[i])

In [8]:
len(data_tens)

60

In [9]:
y_data = [[[[i,j, k] for k in range(poro_tensor.shape[2])]for j in range(poro_tensor.shape[1])] for i in range(poro_tensor.shape[0])]

In [10]:
len(y_data)

60

In [11]:
import itertools

In [12]:
y_data_mem = []
y_1 = []
for i in range(len(y_data)):
  y_1.append(list(itertools.chain(*y_data[i])))

y_to_pd = list(itertools.chain(*y_1))

In [13]:
len(y_to_pd)

1122000

In [14]:
np.array(y_to_pd).shape

(1122000, 3)

In [15]:
np.reshape(poro_tensor.flatten(),[poro_tensor.flatten().shape[0],1]).shape

(1122000, 1)

In [16]:
np.array(y_to_pd)[0][0]

0

In [17]:
i_mem = []
j_mem = []
k_mem = []
for i in range(np.array(y_to_pd).shape[0]):
  i_mem.append(np.float32(y_to_pd[i][0]))
  j_mem.append(np.float32(y_to_pd[i][1]))
  k_mem.append(np.float32(y_to_pd[i][2]))

In [18]:
i_mem[0]

0.0

In [19]:
d = {'i': i_mem, 'j': j_mem, 'k': k_mem, 'value': poro_tensor.flatten().tolist()}

In [20]:
D0TA_FRAME = pd.DataFrame(data=d)

In [21]:
# D0TA_FRAME = D0TA_FRAME.loc[0:1000]
D0TA_FRAME

,i,j,k,value
0,0.0,0.0,0.0,0.184778
1,0.0,0.0,1.0,0.191699
2,0.0,0.0,2.0,0.159089
3,0.0,0.0,3.0,0.151273
4,0.0,0.0,4.0,0.123168
...,...,...,...,...
1121995,59.0,219.0,80.0,0.033210
1121996,59.0,219.0,81.0,0.077864
1121997,59.0,219.0,82.0,0.092004
1121998,59.0,219.0,83.0,0.090571


In [22]:
X = D0TA_FRAME[['i','j','k']]
Y = np.float32(D0TA_FRAME['value'])

In [23]:
X.shape

(1122000, 3)

In [24]:
Y[0].dtype

dtype('float32')

In [25]:
import torch

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
DEVICE

device(type='cuda', index=0)

In [27]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(X.to_numpy())
targets = torch.from_numpy(Y)

In [28]:
inputs.shape

torch.Size([1122000, 3])

In [32]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

In [33]:
x_train = torch.FloatTensor(inputs)
y_train = torch.FloatTensor(targets)

In [34]:
inputs.shape

torch.Size([1122000, 3])

In [42]:
model = Feedforward(3, 60000)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [43]:
x_test = torch.FloatTensor(inputs[0:100])
y_test = torch.FloatTensor(targets[0:100])

In [ ]:
model.eval()
y_pred = model(x_train)
before_train = criterion(y_pred.squeeze(), y_train)
print('Test loss before training' , before_train.item())

In [37]:
model.train()
epoch = 100
for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.02728753164410591
Epoch 1: train loss: 0.02665335312485695
Epoch 2: train loss: 0.554000735282898
Epoch 3: train loss: 0.03780006244778633
Epoch 4: train loss: 0.03780006244778633
Epoch 5: train loss: 0.03780006244778633
Epoch 6: train loss: 0.03780006244778633
Epoch 7: train loss: 0.03780006244778633
Epoch 8: train loss: 0.03780005872249603
Epoch 9: train loss: 0.03780005872249603
Epoch 10: train loss: 0.03780005872249603
Epoch 11: train loss: 0.03780005872249603
Epoch 12: train loss: 0.03780005872249603
Epoch 13: train loss: 0.03780005872249603
Epoch 14: train loss: 0.03780005872249603
Epoch 15: train loss: 0.03780005872249603
Epoch 16: train loss: 0.03780005872249603
Epoch 17: train loss: 0.03780005872249603
Epoch 18: train loss: 0.03780005872249603
Epoch 19: train loss: 0.03780005872249603
Epoch 20: train loss: 0.03780005872249603
Epoch 21: train loss: 0.03780005872249603
Epoch 22: train loss: 0.037800054997205734
Epoch 23: train loss: 0.037800054997205734
Ep

In [38]:
model.eval()
y_pred = model(x_train)
after_train = criterion(y_pred.squeeze(), y_train) 
print('Test loss after Training' , after_train.item())

Test loss after Training 0.03780001774430275


In [40]:
print('RMSE: ', round(0.03780001774430275**0.5,3))

RMSE:  0.194
